In [1]:
import pandas as pd
import networkx as nx
import json
import functions.triplets as ft
import importlib
importlib.reload(ft)

ref = pd.read_csv('data/findings/ref.csv')
abstract = pd.read_csv('data/abstract/abstract.csv')
findings = pd.read_csv('data/findings/findings.csv')
merged_keys = pd.read_csv('data/graph/merged_keys.csv')
clustered_keys = pd.read_csv('data/graph/clustered_keys.csv')

In [2]:
for i, row in findings.iterrows():
    findings.at[i, 'key'] = f"{i}:{row['paper-id']}"

In [3]:
triplets = pd.read_csv('data/triplets/triplets.csv')
triplets['cause'] = triplets['cause'].apply(lambda x: json.loads(x))
triplets['effect'] = triplets['effect'].apply(lambda x: json.loads(x))

for t in ['cause', 'effect']:
    triplets[f"{t}_full"] = triplets[t].apply(lambda x: ft.parse_key(x, full=True))
    triplets[f"{t}_short"] = triplets[t].apply(lambda x: ft.parse_key(x))

In [4]:
findings

,paper-id,finding,key
0,paper-0,The AI reconstruction system improves face rep...,0:paper-0
1,paper-0,The AI system achieves a 60.6% identification ...,1:paper-0
2,paper-1,The AI sensor creation system enhances user co...,2:paper-1
3,paper-1,The AI sensor creation system uncovers overloo...,3:paper-1
4,paper-2,The AI conversational explanation system EMCEE...,4:paper-2
...,...,...,...
1859,paper-1339,The AI system's potential is constrained by un...,1859:paper-1339
1860,paper-1342,The AI ethics design tool clarifies difference...,1860:paper-1342
1861,paper-1342,The AI ethics design tool recommends scenario ...,1861:paper-1342
1862,paper-1346,AI-based virtual influencers drive usage inten...,1862:paper-1346


In [5]:
triplets['ref-id'] = triplets['paper-id'].apply(lambda x: ref[ref['ref_id'] == x.split(":")[1]].iloc[0]['id'])
triplets['paper'] = triplets['ref-id'].apply(lambda x: abstract[abstract['paperId'] == x].iloc[0]['title'])
triplets['finding'] = triplets['paper-id'].apply(lambda x: findings[findings['key'] == x].iloc[0]['finding'])

triplets.drop(columns=["ref-id", "cause", "effect"])

,paper-id,relation,cause_full,cause_short,effect_full,effect_short,paper,finding
0,0:paper-0,INCREASES,ai|reconstruction>system,ai|reconstruction,co|representation>accuracy:face,co|representation,HAIFAI: Human-AI Interaction for Mental Face R...,The AI reconstruction system improves face rep...
1,1:paper-0,INCREASES,ai>reconstruction:performance,ai>reconstruction,co|metric>identification:rate,co|metric,HAIFAI: Human-AI Interaction for Mental Face R...,The AI system achieves a 60.6% identification ...
2,2:paper-1,INCREASES,ai|sensor>personalized_definition:criteria,ai|sensor,human|user>control,human|user,Gensors: Authoring Personalized Visual Sensors...,The AI sensor creation system enhances user co...
3,3:paper-1,INCREASES,ai|sensor>creation,ai|sensor,co|criterion>overlooked,co|criterion,Gensors: Authoring Personalized Visual Sensors...,The AI sensor creation system uncovers overloo...
4,4:paper-2,INCREASES,ai|conversational>explanation,ai|conversational,co|score>automatic:evaluation,co|score,Conversational Explanations: Discussing Explai...,The AI conversational explanation system EMCEE...
...,...,...,...,...,...,...,...,...
1853,1859:paper-1339,DECREASES,co|tension>data_privacy:public_health,co|tension,ai>potential,ai>potential,Artificial intelligence vs COVID-19: limitatio...,The AI system's potential is constrained by un...
1854,1860:paper-1342,INCREASES,ai|tool>ethics_design,ai|tool,co|structure>difference:semantic,co|structure,Organic and dynamic tool for use with knowledg...,The AI ethics design tool clarifies difference...
1855,1861:paper-1342,INCREASES,ai|tool>recommendation:scenario,ai|tool,human|engineer>integration:ethics,human|engineer,Organic and dynamic tool for use with knowledg...,The AI ethics design tool recommends scenario ...
1856,1862:paper-1346,INFLUENCES,ai|virtual_influencer>presence,ai|virtual_influencer,co|mechanism>compensatory,co|mechanism,“Threatened and empty selves following AI-base...,AI-based virtual influencers drive usage inten...


In [6]:
import ast
merged_map = {}
for i, row in merged_keys.iterrows():
    for k in ast.literal_eval(row['members']):
        merged_map[k] = row['representative']

In [7]:
# clustered_keys
len(set(triplets['cause_short'].apply(lambda x: x if x.split(">")[0] in ['human', 'ai', 'co'] else None).dropna().to_list()))

71

In [ ]:
G = nx.MultiDiGraph()
reps = merged_keys['representative'].to_list()

for cluster in [*clustered_keys['cluster'].unique().tolist(), 'human', 'ai', 'co']:
    G.add_node(
        cluster,
        kind='cluster',
        description=cluster
    )

for i, k in clustered_keys.iterrows():
    members = ast.literal_eval(merged_keys.loc[merged_keys['representative'] == k['key']].iloc[0]['members'])  if k['key'] in reps else [k['key']]
    G.add_node(
        k['key'],
        cluster=k['cluster'],
        kind='key',
        members=members
    )
    G.add_edge(
        k['cluster'], k['key'],
        kind='cluster'
    )
    
for i, f in triplets.iterrows():
    # if(not f['effect_short'] in G.nodes and not f['effect_short'] in merged_map):
    #     print(f['effect_short'])

    source = merged_map[f['cause_short']] if f['cause_short'] in merged_map else f['cause_short']
    target = merged_map[f['effect_short']] if f['effect_short'] in merged_map else f['effect_short']
    G.add_edge(
        source, target,
        kind="relation",
        source_full=f['cause_full'],
        effect_full=f['effect_full'],
        type=f['relation'],
        paper=f['paper'],
        finding=f['finding']
    )

In [9]:
import json
graph_data = nx.node_link_data(G, edges="edges")
with open('data/graph/graph.json', 'w') as f:
    f.write(json.dumps(graph_data))

In [10]:
merged_keys['representative'].to_list()

['co|partner',
 'ai|sentiment',
 'ai|medical',
 'co|organizational',
 'human|writer',
 'human|demographic',
 'ai>integration',
 'human>frustration',
 'ai|automated',
 'ai|simulation',
 'ai|prompt',
 'human>#perception',
 'ai|fine_tuning',
 'human|crowd_worker',
 'co|design_ideation',
 'ai|heuristics',
 'ai|vision_language',
 'ai|gpt4',
 'ai|algorithmic',
 'ai|decision_aid',
 'ai|finance',
 'ai|codegeneration',
 'co|psychological',
 'human>#confidence',
 'ai|diagnostic',
 'ai|gameplay',
 'co|explainability_design',
 'ai|symptomchecker',
 'human|accountant',
 'ai|writing',
 'human>cognition',
 'ai|generated',
 'human>judgment',
 'co|guideline',
 'ai|virtual_agent',
 'co|coding',
 'ai|deeplearning',
 'ai|narrative',
 'ai|coding',
 'ai|neural_network',
 'ai|self_driving',
 'human|teacher',
 'ai|recommendation',
 'ai|chatspot',
 'ai|personalization',
 'co|game',
 'co|cultural',
 'co|politics',
 'human|decisionmaker',
 'ai|transparent',
 'ai|control',
 'human>preference',
 'human|comicprofes